### Урок 4. Парсинг HTML. XPath

* Выберите веб-сайт с табличными данными, который вас интересует.
* Напишите код Python, использующий библиотеку requests для отправки HTTP GET-запроса на сайт и получения HTML-содержимого страницы.
* Выполните парсинг содержимого HTML с помощью библиотеки lxml, чтобы извлечь данные из таблицы.
* Сохраните извлеченные данные в CSV-файл с помощью модуля csv.

Ваш код должен включать следующее:

- Строку агента пользователя в заголовке HTTP-запроса, чтобы имитировать веб-браузер и избежать блокировки сервером.
- Выражения XPath для выбора элементов данных таблицы и извлечения их содержимого.
- Обработка ошибок для случаев, когда данные не имеют ожидаемого формата.
- Комментарии для объяснения цели и логики кода.

Целью данной работы будет попытка получить данные по турнирам настольного тенниса от Лига-Про.

1. Пытаемся получить ответ с get-запроса по интересующему нас дню
2. Получить информацию по этому дню о предстоящих турнирах со списком участников
3. Сформировать из полученных данных датафрейм
4. Сохранить данных в табличном формате в файле csv

In [1]:
#!pip install lxml

In [2]:
# импортируем нужные библиотеки
from lxml import html
import requests
from pprint import pprint
import csv
import pandas as pd

In [3]:
url='https://tt.sport-liga.pro/'

# задаем user-agent
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }
# в params надо указать интересующую нас дату в формате YYYY:MM:DD
# param = {
#     'year': '2024',
#     'month': '05',
#     'day': '7'
# }

url_tour = 'tours/'
session = requests.Session()

In [4]:
# просьба ввести данные пользователя из консоли (В РАЗРАБОТКЕ)
date = input('Введите дату в формате DD:MM:YYYY ')
day, month, year = date.split('.')
params = {
    'year': year,
    'month': month,
    'day': day
}

In [5]:
# делаем get-запрос
response = session.get(url+url_tour, params=params, headers=header)
# проверяем статус-код
if response.status_code < 400:
    # если да, печатаем статус код и представляем ответ в виде дерева html
    print(response.status_code)
    tree = html.fromstring(response.text)
else:
    print('Что-то пошло не так')

200


Необходимые данные находятся на странице в нижней таблице. Также для сбора фамилий участников турнира нам нужно переходить на его страницу (ее можно "достать" из таблицы.)

In [6]:
# находим нужную таблицу на странице
table = tree.xpath('//table[@class="table"]//tr')
# находим ссылки на переход к просмотру турнира
ref = tree.xpath('.//td[@class="tournament-name"]/a/@href')

Для объединения всех данных в один список будем проходиться циклом for сразу по двум спискам, для этого объединим их функцией zip

In [7]:
tournaments = []
for row, item in zip(table[1:], ref):
    # сделаем get-запрос на страницу турнира
    res = session.get(url+item, headers=header)
    # и получим дерево из его ответа
    tour = html.fromstring(res.text)
    # создадим словарик для каждого турнира
    tours_info = {}
    # из даты нам нужно только время начало турнира
    time = row.xpath('.//td[@class="tournament-date"]/text()')
    time = [el.split()[-1] for el in time][0]

    # из названия турнира мы возьмем
    tours = row.xpath('.//td[@class="tournament-name"]/a/text()')
    # название стола
    tables = [el.split()[1][:-1] for el in tours][0]
    # уровень лиги
    liga = [el.split()[3] for el in tours][0]

    # участников турнира получим из страницы турнира
    parties = tour.xpath('//td[@class="nowrap"]/a/text()')

    tours_info['date'] = date
    tours_info['time'] = time
    tours_info['tables'] = tables
    tours_info['liga'] = liga
    tours_info['parties'] = parties
    
    # добавим каждый словарь в список
    tournaments.append(tours_info)


In [8]:
pprint(tournaments)

[{'date': '13.05.2024',
  'liga': '500-550',
  'parties': ['Илюхин Е', 'Кустов Д', 'Евлахин Д', 'Пандур И', 'Денисов А-й'],
  'tables': 'А6',
  'time': '23:45'},
 {'date': '13.05.2024',
  'liga': '350-400',
  'parties': ['Идрисов А',
              'Харлакин О',
              'Немашкало В',
              'Мареев Н',
              'Янковский А'],
  'tables': 'А4',
  'time': '23:45'},
 {'date': '13.05.2024',
  'liga': '450-500',
  'parties': ['Тихненко Д', 'Попов О', 'Зюганов О', 'Мамека М', 'Барышников Е'],
  'tables': 'А5',
  'time': '23:30'},
 {'date': '13.05.2024',
  'liga': '700-800',
  'parties': ['Юдин А', 'Морозов С', 'Лисов А', 'Молодых А'],
  'tables': 'А9',
  'time': '20:00'},
 {'date': '13.05.2024',
  'liga': '800-900',
  'parties': ['Рахматов А', 'Баталов А', 'Кочетков К', 'Евлампиев В'],
  'tables': 'А3',
  'time': '20:00'},
 {'date': '13.05.2024',
  'liga': '500-550',
  'parties': ['Поляков П', 'Помазков А', 'Прохоров А', 'Мягков С'],
  'tables': 'А6',
  'time': '19:45'},
 

Запишем полученную информацию в csv-файл

In [9]:
with open('./tournaments.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=tournaments[0].keys())
    writer.writeheader
    for row in tournaments:
        writer.writerow(row)
print('Данные успешно записаны')

Данные успешно записаны


Для наглядности представим наш список в виде датафрейма

In [14]:
df = pd.DataFrame(tournaments)
df

,date,time,tables,liga,parties
0,13.05.2024,23:45,А6,500-550,"[Илюхин Е, Кустов Д, Евлахин Д, Пандур И, Дени..."
1,13.05.2024,23:45,А4,350-400,"[Идрисов А, Харлакин О, Немашкало В, Мареев Н,..."
2,13.05.2024,23:30,А5,450-500,"[Тихненко Д, Попов О, Зюганов О, Мамека М, Бар..."
3,13.05.2024,20:00,А9,700-800,"[Юдин А, Морозов С, Лисов А, Молодых А]"
4,13.05.2024,20:00,А3,800-900,"[Рахматов А, Баталов А, Кочетков К, Евлампиев В]"
5,13.05.2024,19:45,А6,500-550,"[Поляков П, Помазков А, Прохоров А, Мягков С]"
6,13.05.2024,19:45,А4,550-600,"[Солдатов И, Иванов Д, Никулин С, Сухарников Н]"
7,13.05.2024,19:30,А5,900-1000,"[Лукьянов П, Дедов В, Пузанков К, Сташковский Г]"
8,13.05.2024,16:00,А3,250-300,"[Цветков Э, Хургин В, Смирнов М, Попроцкий А]"
9,13.05.2024,15:45,А4,400-450,"[Демченко В, Мухамеджанов М, Руденко В, Алексе..."


Вывод: В результате проделанной работы мы получаем датафрейм, содержащий нужную информацию. Этот датафрейм планируется использовать для подбора турнира с более предпочтительными игроками.

Также планируется исследовать каждый турнир на предмет статистики личных встреч между игроками

In [16]:
#df = df[df['tables'] != 'А15']
df[df['time'] > '15:00']

,date,time,tables,liga,parties
0,13.05.2024,23:45,А6,500-550,"[Илюхин Е, Кустов Д, Евлахин Д, Пандур И, Дени..."
1,13.05.2024,23:45,А4,350-400,"[Идрисов А, Харлакин О, Немашкало В, Мареев Н,..."
2,13.05.2024,23:30,А5,450-500,"[Тихненко Д, Попов О, Зюганов О, Мамека М, Бар..."
3,13.05.2024,20:00,А9,700-800,"[Юдин А, Морозов С, Лисов А, Молодых А]"
4,13.05.2024,20:00,А3,800-900,"[Рахматов А, Баталов А, Кочетков К, Евлампиев В]"
5,13.05.2024,19:45,А6,500-550,"[Поляков П, Помазков А, Прохоров А, Мягков С]"
6,13.05.2024,19:45,А4,550-600,"[Солдатов И, Иванов Д, Никулин С, Сухарников Н]"
7,13.05.2024,19:30,А5,900-1000,"[Лукьянов П, Дедов В, Пузанков К, Сташковский Г]"
8,13.05.2024,16:00,А3,250-300,"[Цветков Э, Хургин В, Смирнов М, Попроцкий А]"
9,13.05.2024,15:45,А4,400-450,"[Демченко В, Мухамеджанов М, Руденко В, Алексе..."
